In [ ]:
import numpy as np   

# Step 1: Compute the gradient 

def gradient_fd(f, x, h=1e-6):
    """
    [Parameters]
    f : The function we want to minimize. Takes a vector x as input and returns a scalar.
    x : The point where we want to approximate the gradient.
    h : A tiny step size for the finite difference calculation.

    [Returns]
    g : Approximation of the gradient at point x.
    """

    x = np.asarray(x, dtype=float)     
    n = x.size                         
    g = np.zeros(n, dtype=float)       

    for i in range(n):
        # ei is a "unit vector" with a 1 in the i-th position and 0 elsewhere.
        ei = np.zeros(n); ei[i] = 1.0

        # Evaluate f slightly forward and backward in the i-th direction.
        f_plus  = f(x + h*ei)
        f_minus = f(x - h*ei)

        # Central difference formula: (f(x+h) - f(x-h)) / (2h)
        g[i] = (f_plus - f_minus) / (2.0*h)

    return g


# Step 2: Compute the matrix of 2nd derivatives

def hessian_fd(f, x, h=1e-5):
    """
    Compute n x n matrix where entry (i,j) is the second derivative 
    of f with respect to variable i and j.

    [Parameters]
    f : Objective function.
    x : Point where Hessian is approximated.
    h : Step size for finite differences.

    [Returns[
    H : Approximate Hessian matrix.
    """

    x = np.asarray(x, dtype=float)
    n = x.size
    H = np.zeros((n, n), dtype=float)   # Initialize an n×n matrix with zeros.
    fx = f(x)                           # f(x) at the current point

    # --- Step 1: Compute diagonal elements (second derivative wrt same variable) ---
    for i in range(n):
        ei = np.zeros(n); ei[i] = 1.0
        f_plus  = f(x + h*ei)
        f_minus = f(x - h*ei)

        # Formula: (f(x+h) - 2f(x) + f(x-h)) / h^2
        H[i, i] = (f_plus - 2.0*fx + f_minus) / (h*h)

    # --- Step 2: Compute off-diagonal elements (mixed partial derivatives) ---
    for i in range(n):
        ei = np.zeros(n); ei[i] = 1.0
        for j in range(i+1, n):   # Only compute upper triangle, then copy to lower triangle (symmetry)
            ej = np.zeros(n); ej[j] = 1.0

            f_pp = f(x + h*ei + h*ej)   # f(x+h in both i and j)
            f_pm = f(x + h*ei - h*ej)   # f(x+h in i, -h in j)
            f_mp = f(x - h*ei + h*ej)   # f(-h in i, +h in j)
            f_mm = f(x - h*ei - h*ej)   # f(-h in both i and j)

            # Formula: (f_pp - f_pm - f_mp + f_mm) / (4 h^2)
            val = (f_pp - f_pm - f_mp + f_mm) / (4.0*h*h)

            # Hessian is symmetric, so H[i,j] = H[j,i]
            H[i, j] = H[j, i] = val

    return H